# Humid Heat Metrics

## Overview

Humid heat metrics index temperature and humidity, which is often more useful than comparing either variable alone when considering perceived temperature, as they together affect the human body's ability to cool itself down. Humid heat metrics are especially important for the safety of outside workers, the elderly, or otherwise high-risk, high-exposure individuals{footcite}`li_2020`.

Wet Bulb Globe Temperature ({math}`WBGT`) is a measure of heat stress. The equations for outdoor ({math}`WBGT_{od}`) and indoor/shaded ({math}`WBGT_{id}`) WBGTs are{footcite}`li_2020`:

{math}`WBGT_{od} = 0.7*T_{nwb} + 0.2*T_g + 0.1*T_a`

{math}`WBGT_{id} = 0.7*T_{nwb} + 0.3*T_g`

where {math}`T_a` refers to Dry Bulb Ambient Temperatue, {math}`T_{nwb}` is the Natural Wet Bulb Temperature with exposure to wind and sun, and {math}`T_g` is the Globe Temperature taken from inside a copper globe painted black and exposed to the sun{footcite}`li_2020`.

However, this formula is complicated by the reality that Natural Wet Bulb Temperature and Globe Temperature are not always readily available variables from weather stations or atmospheric models.

In this notebook, we will demonstrate the Australian Bureau of Meteorology (ABM) and Bernard methods of predicting wet bulb global temperature with a focus on the July 1995 Chicago heatwave.

For our analysis, we have ERA5 reanalysis data for the lower contiguous United States (50&deg;N, 24&deg;S, -66&deg;E, -125&deg;W) from July 1995 with the variables: 2-meter temperature, 2-meter dew point temperature, surface pressure, and *u*/*v* wind components.

ERA5 is a reanalysis, a global weather/climate dataset that combines model output and observations with physical understanding to create a spatially and temporally consistent historic dataset, spanning from 1940 to today (updated every 5 days){footcite}`era5`.

---

In [ ]:
import geocat.datafiles as gdf
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from scipy import optimize

In [ ]:
era5 = xr.open_dataset(gdf.get("netcdf_files/era5_1995-07-14T12.nc"))
era5.u10

In [3]:
# Convert from Kelvin to Celsius

era5['t2m_C'] = era5['t2m'] - 273.15
era5['d2m_C'] = era5['d2m'] - 273.15

In [4]:
# Chicago coordinates
lat_chicago = 41.8781
lon_chicago = -87.6298

## Australian Bureau of Meteorology (ABM)

The [Australian Bureau of Meteorology](http://www.bom.gov.au/)'s method of estimating Wet Bulb Global Temperature is attractive due to its simplicity. It only requires temperature and relative humidity{footcite}`abm_2010`.

Below is a chart of WBGT from relative humidity and temperature{footcite}`abm_2010`:

<img src="../_static/images/wbgt_approximation.png">

This method tends to overpredict WBGT compared to other models and assumes full sunlight and light breeze.

In [5]:
def calc_abm_wbgt(t_a, rh):
    p = (
        (rh / 100) * 6.105 * np.exp(17.27 * t_a / (237.7 + t_a))
    )  # water vapor pressure [hPa]
    wbgt = (0.567 * t_a) + (0.393 * p) + 3.94
    return wbgt

To use our ERA5 data in this equation, we need to first calculate relative humidity (the ratio of vapor pressure to saturation pressure) from temperature and dewpoint. To do this we use the Magnus-Tetens Approximation for vapor pressure{footcite}`monteith_2008`:

{math}`e = 6.11 \exp {\left( \frac{17.625 \times t}{t + 243.04} \right)}`

where {math}`e` is vapor pressure and {math}`t` is temperature in Kelvin.

In [6]:
def _calc_vapor_pressure(t):  # Magnus-Tetens Approximation
    e = 6.11 * np.exp((17.27 * t) / (t + 237.3))  # Vapor Pressure in hPa
    return e


def calc_relative_humidity_era5(t_a, t_d):
    e = _calc_vapor_pressure(t_d)  # vapor pressure from dew point temp
    e_sat = _calc_vapor_pressure(t_a)  # saturation vapor pressure
    rh = 100 * e / e_sat  # Clausius-Clapeyron equation
    return rh


rh = calc_relative_humidity_era5(era5.t2m_C, era5.d2m_C)

In [ ]:
wbgt_abm = calc_abm_wbgt(era5.t2m_C, rh)
wbgt_abm

### Plotting Chicago ABM WBGT

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

c = plt.contourf(era5.longitude, era5.latitude, wbgt_abm, cmap='inferno')

ax.coastlines()

cbar = plt.colorbar(c, ax=ax, orientation='horizontal')
cbar.set_label('Wet Bulb Global Temperature' + '\N{DEGREE SIGN}' + 'C')

ax.set_title('July 14, 1995 noon - ABM WBGT')

# Annotate location of Chicago
ax.plot(lon_chicago, lat_chicago, 'k*');

## Bernard

Bernard's semi-empirical formula approximates Natural Wet Bulb temperature based on heat exchange of a wetted wick exposed to sun and wind based on measurements of common United States summertime environmental conditions{footcite}`bernard_1999`.

This is considered an indoor WBGT temperature because it does not include any strong radiative sources in the calculation.

{math}`
WBGT_{id} = \begin{cases} 
      0.7T_{pwb} + 0.3T_a & \text{if } v > 3 m/s\\
      0.67T_{pwb} + 0.33T_a − 0.048 log_10v (T_a − T_{pwb}) & \text{if } 0.3 \geq v \leq 3 m/s 
   \end{cases}
`

In Bernard's analysis, wind speeds less than 0.3 m/s are not included since the field of humid heat metrics is primarily concerned with workers, and an outdoor worker is unlikely to be stationary. Apparent wind speeds are assumed to be at least 1 m/s{footcite}`bernard_1999`.

This formula utilizes thermodynamic Wet Bulb Temperature ({math}`T_{pwb}`), which is a wet bulb temperature in the shade and fanned or rotated. This is the wet bulb typically used for dew point calculations, and can be iteratively derived from temperature ({math}`T_a`) and dewpoint ({math}`T_d`).

In [9]:
# Bernard formula for WBGT
def calc_bernard_wbgt(t_a, t_pwb, v):
    if np.all(v < 0.3):  # m/s
        return np.nan  # Return NaN where velocity is below the threshold
    elif np.all((0.3 <= v) & (v <= 3)):
        wbgt = (0.67 * t_pwb) + (0.33 * t_a) - (0.48 * np.log10(v) * (t_a - t_pwb))
    else:
        wbgt = (0.7 * t_pwb) + (0.3 * t_a)

    return wbgt

$T_{pwb}$ is iteratively solved from McPherson's formula{footcite}`lemke_kjellstrom_2012`:

{math}`1556 e_d - 1.484 e_d * T_{pwb} - 1556 e_w + 1.484 * e_w * T_{pwb} + 1010 * (t_a - t_pwb) = 0`

where {math}`e_d = 6.106 * exp(17.27 * T_d / (237.3 + T_d))` \
and {math}`e_w = 6.106 * exp(17.27 * T_{pwb} / (237.3 + T_{pwb}))`

Here we use a Newton-Raphson iterative method for the iterative solve for {math}`t_{pwb}`. 

{math}`x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}`

Essentially, Newton-Raphson is a root finding method that plugs your initial guess into the equation in question and the derivative of that equation in order to get a more accurate guess {footcite}`atkinson_1989`. This is repeated until your new guess is sufficiently close. Thankfully we have [`scipy.optimize.newton()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html) to handle this solve for us.

In [10]:
# Scalar function to compute t_pwb
def _calc_tpwb_scalar(t_a, t_d):
    def f(t_pwb):
        e_d = 6.106 * np.exp(17.27 * t_d / (237.3 + t_d))  # hPa
        # e_w = 6.106 * np.exp(17.27 * t_pwb / (237.3 + t_pwb))
        func = (
            1556 * e_d
            - 1.484 * e_d * t_pwb
            - 1556 * (6.106 * np.exp(17.27 * t_pwb / (237.3 + t_pwb)))
            + 1.484 * (6.106 * np.exp(17.27 * t_pwb / (237.3 + t_pwb))) * t_pwb
            + 1010 * (t_a - t_pwb)
        )
        return func

    def f_prime(t_pwb, h=1e-5):  # numerical derivative
        return (f(t_pwb + h) - f(t_pwb - h)) / (2 * h)

    # Use the Newton-Raphson method with scipy's newton function
    t_pwb_0 = t_d  # initial guess
    t_pwb = optimize.newton(f, t_pwb_0, fprime=f_prime, tol=1e-6, maxiter=100)
    return t_pwb


# Apply function over grid
def _calc_tpwb(t_a, t_d):
    return xr.apply_ufunc(
        _calc_tpwb_scalar,
        t_a,
        t_d,
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float],
    )

In [ ]:
v = np.sqrt(era5.u10**2 + era5.v10**2)  # combine u and v wind components
t_pwb = _calc_tpwb(era5.t2m_C, era5.d2m_C)

wbgt_bernard = calc_bernard_wbgt(era5.t2m_C, t_pwb, v)
wbgt_bernard

### Plotting Chicago Bernard WBGT

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

c = plt.contourf(era5.longitude, era5.latitude, wbgt_bernard, cmap='inferno')

ax.coastlines()

cbar = plt.colorbar(c, ax=ax, orientation='horizontal')
cbar.set_label('Wet Bulb Global Temperature' + '\N{DEGREE SIGN}' + 'C')

ax.set_title('July 14, 1995 noon - Bernard WBGT method')

# Annotate location of Chicago
ax.plot(lon_chicago, lat_chicago, 'k*');

## Comparing methods

When comparing our output from both ABM and Bernard, ABM tends to estimate at higher WBGT by 4 - 7 degrees Celsius.

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

diff = wbgt_abm - wbgt_bernard

c = plt.contourf(era5.longitude, era5.latitude, diff, cmap='Reds')

cbar = plt.colorbar(c, ax=ax, orientation='horizontal')
cbar.set_label(
    '\N{GREEK CAPITAL LETTER DELTA} Wet Bulb Global Temperature'
    + '\N{DEGREE SIGN}'
    + 'C'
)
ax.set_title('July 14, 1995 noon - WBGT Difference (ABM - Bernard)')

# Annotate location of Chicago
ax.plot(lon_chicago, lat_chicago, 'k*');

## References:

```{footbibliography}
```